In [13]:
import torch
import gdown
import pandas as pd
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn
import os

In [14]:
df = pd.read_csv("AustinHousingData_Preprocessed.csv")

In [15]:
# Prikaz sadržaja ćelije u koloni homeImage i price za prvi red
image_url_first_row = df.loc[0, 'homeImage']
print("Sadržaj ćelije u koloni homeImage za prvi red:", image_url_first_row)
price_first_row = df.loc[0, 'price']
print("Vrednost u koloni price za prvi red:", price_first_row)

Sadržaj ćelije u koloni homeImage za prvi red: 116080992_83c089e97b4e96ecb01c3555fa8252d0-p_f.jpg
Vrednost u koloni price za prvi red: 689225.7


In [16]:
image_url_first_row = df.loc[1, 'homeImage']
print("Sadržaj ćelije u koloni homeImage za drgi red:", image_url_first_row)
price_first_row = df.loc[1, 'price']
print("Vrednost u koloni price za drugi red:", price_first_row)

Sadržaj ćelije u koloni homeImage za drgi red: 88013985_6d4b35d2a232c42fcc863317f22401c7-p_f.jpg
Vrednost u koloni price za drugi red: 409818.0


In [17]:
image_url_first_row = df.loc[2, 'homeImage']
print("Sadržaj ćelije u koloni homeImage za treci red:", image_url_first_row)
price_first_row = df.loc[2, 'price']
print("Vrednost u koloni price za treci red:", price_first_row)

Sadržaj ćelije u koloni homeImage za treci red: 70344594_3e29ab1c54a4a8ca80360911b9ac7434-p_f.jpg
Vrednost u koloni price za treci red: 203229.6832022083


In [20]:
image_url_first_row = df.loc[3, 'homeImage']
print("Sadržaj ćelije u koloni homeImage za cetvrti red:", image_url_first_row)
price_first_row = df.loc[3, 'price']
print("Vrednost u koloni price za cetvrti red:", price_first_row)

Sadržaj ćelije u koloni homeImage za cetvrti red: 185217484_1f06634bc9d5424bc0f094f68ca72fe4-p_f.jpg
Vrednost u koloni price za cetvrti red: 408501.87578333326


In [21]:
image_url_first_row = df.loc[4, 'homeImage']
print("Sadržaj ćelije u koloni homeImage za peti red:", image_url_first_row)
price_first_row = df.loc[4, 'price']
print("Vrednost u koloni price za peti red:", price_first_row)

Sadržaj ćelije u koloni homeImage za peti red: 116080981_b15ecd54b82b93b6af1408beb22e069f-p_f.jpg
Vrednost u koloni price za peti red: 550999.8


In [22]:
class ResNetRegression(nn.Module):
    def __init__(self, input_channels=3):
        super(ResNetRegression, self).__init__()
        resnet = models.resnet50(pretrained=True)
        # Modify the first convolutional layer to match the number of input channels
        resnet.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        resnet.fc = nn.Sequential(
            nn.Linear(resnet.fc.in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.resnet = resnet

    def forward(self, x):
        return self.resnet(x)


In [ ]:
# Parametri za denormalizaciju
min_original_price = 79402.0705
max_original_price = 1954899.0775

In [23]:
# Funkcija za denormalizaciju cene
def denormalize_price(normalized_price, min_original_price, max_original_price):
    return normalized_price * (max_original_price - min_original_price) + min_original_price

In [24]:
# Direktni link za preuzimanje
google_drive_url = 'https://drive.google.com/uc?id=1JoAJGKQRB9itW5U_xkc-QGnkNMHb_0vu'

# Preuzimanje modela sa Google Drive-a
output_path = 'Price-Image-Predictor.pth'
gdown.download(google_drive_url, output_path, quiet=False)

# Učitavanje modela kao OrderedDict
state_dict = torch.load(output_path)
model = ResNetRegression()  # Zamijenite ResNetRegression sa stvarnom klasom vašeg modela
model.load_state_dict(state_dict)
model.eval()

# Direktorijum u kojem se nalazi Jupyter notebook
current_directory = os.getcwd()

# Lista slika sa ekstenzijom .jpg u trenutnom direktorijumu
image_paths = [f for f in os.listdir(current_directory) if f.endswith('.jpg')]


# Iteriranje kroz listu slika
for img_path in image_paths:
    # Učitavanje i priprema slike
    img = Image.open(img_path)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    img_tensor = transform(img)
    img_tensor = img_tensor.unsqueeze(0)

    # Predviđanje cene
    with torch.no_grad():
        output = model(img_tensor)

    # Normalizovana cena koju smo dobili iz modela
    normalized_price = output.item()

    # Denormalizacija
    denormalized_price = denormalize_price(normalized_price, min_original_price, max_original_price)

    # Ispis denormalizovane cene za svaku sliku
    print(f"Denormalizovana cena za {img_path}: {denormalized_price}")

Downloading...
From: https://drive.google.com/uc?id=1JoAJGKQRB9itW5U_xkc-QGnkNMHb_0vu
To: /content/Price-Image-Predictor.pth
100%|██████████| 95.4M/95.4M [00:01<00:00, 78.1MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Denormalizovana cena za 88013985_6d4b35d2a232c42fcc863317f22401c7-p_f.jpg: 330270.2926949668
Denormalizovana cena za 70344594_3e29ab1c54a4a8ca80360911b9ac7434-p_f.jpg: 334088.6747282166
Denormalizovana cena za 116080981_b15ecd54b82b93b6af1408beb22e069f-p_f.jpg: 334417.4162681321
Denormalizovana cena za 185217484_1f06634bc9d5424bc0f094f68ca72fe4-p_f.jpg: 328412.14702584303
Denormalizovana cena za 116080992_83c089e97b4e96ecb01c3555fa8252d0-p_f.jpg: 335303.5064887018
